Project Name: Analysis the distribution of Chinese Dialects among small datasets
Authors:     Jiarong Tang      Weiting Wang 
Descripition: Use the subword embbedings(FastText), string distance, network tree and language models to show the distribution of Chinese dialects.

In [1]:
import pandas as pd


In [2]:
data_df = pd.read_csv('forms.csv')
data_df

,ID,Local_ID,Language_ID,Parameter_ID,Value,Form,Segments,Comment,Source,Cognacy,Loan,Graphemes,Profile,Prosody,Morpheme_Glosses,Partial_Cognacy,Chinese_Characters
0,Beijing-91_vomit-1,NaN,Beijing,91_vomit,tʰu⁵¹,tʰu⁵¹,tʰ u ⁵¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
1,Haerbin-91_vomit-1,NaN,Haerbin,91_vomit,tʰu⁵³,tʰu⁵³,tʰ u ⁵³,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
2,Jinan-91_vomit-1,NaN,Jinan,91_vomit,tʰu³¹,tʰu³¹,tʰ u ³¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
3,Rongcheng-91_vomit-1,NaN,Rongcheng,91_vomit,ou²¹³⁻³⁵ tʰu²¹⁴,ou²¹³⁻³⁵ tʰu²¹⁴,ou ²¹³ + tʰ u ²¹⁴,copulative synonyme,Liu2007,NaN,NaN,NaN,NaN,n t + i n t,nausea/嘔 spit/吐,2 1,嘔 吐
4,Taiyuan-91_vomit-1,NaN,Taiyuan,91_vomit,tʰu⁵³ lə⁰,tʰu⁵³ lə⁰,tʰ u ⁵³ + l ə ⁰,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t,nausea/嘔 _:PERFECTIVE/了,2 5,嘔 嘞
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4297,Guangzhou-90_woman-1,NaN,Guangzhou,90_woman,nøy²³ iɐn²¹⁻²³,nøy²³ iɐn²¹⁻²³,n øy ²³ + j ɐ n ²¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n c t,female/女 _person/人,39 38,女 人
4298,Fuzhou-90_woman-1,NaN,Fuzhou,90_woman,i⁵⁵⁻⁵³ tsia³²,i⁵⁵⁻⁵³ tsia³²,i ⁵⁵ + ts j a ³²,NaN,Liu2007,NaN,NaN,NaN,NaN,n t + i m n t,woman/伊 sister/姐,824 715,伊 姐
4299,Fuzhou-90_woman-2,NaN,Fuzhou,90_woman,ny³²⁻⁵⁵ ɛ²¹²,ny³²⁻⁵⁵ ɛ²¹²,n y ³² + ɛ ²¹²,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + n t,female/女 _world/界,39 825,女 界
4300,Fuzhou-90_woman-3,NaN,Fuzhou,90_woman,tsy⁵⁵ nøyŋ⁵³⁻⁵⁵ nøyŋ⁵³,tsy⁵⁵ nøyŋ⁵³⁻⁵⁵ nøyŋ⁵³,ts y ⁵⁵ + n øy ŋ ⁵³ + n øy ŋ ⁵³,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n c t + i n c t,woman/諸 female/娘 _person/人,749 31 38,諸 娘 儂


step 1 extract all the parameter_ID/ Language_ID to decide how many words we are inverstigating


In [3]:
# 19 cities  but there is variety among the same city 
cities = data_df['Language_ID'].unique()
cities

array(['Beijing', 'Haerbin', 'Jinan', 'Rongcheng', 'Taiyuan', 'XiAn',
       'Chengdu', 'Nanjing', 'Jixi', 'Suzhou', 'Wenzhou', 'Changsha',
       'Loudi', 'Nanchang', 'Meixian', 'Guilin', 'Guangzhou', 'Fuzhou',
       'Xiamen'], dtype=object)

In [4]:
# 203 words 
words = data_df['Parameter_ID'].unique()
words

array(['91_vomit', '92_fear', '93_skin', '94_float', '95_smooth',
       '96_wife', '97_all', '98_hot', '99_person', '100_throw', '1_white',
       '2_back', '3_nose', '4_ice', '5_neck', '6_thin', '7_not', '8_rub',
       '9_grass', '10_long', '101_flesh', '102_if', '103_breasts',
       '104_three', '105_woods', '106_kill', '107_sand', '108_mountain',
       '109_burn', '110_few', '111_tongue', '112_snake', '113_rope',
       '114_louse', '115_wet', '116_what', '117_stone', '118_hand',
       '119_tree', '120_bark', '121a_countnoun', '121b_countverb',
       '122_who', '123_water', '124_fruit', '125_sleep', '126_suck',
       '127_say', '128_split', '129_die', '130_four', '131_he',
       '132_they', '133_sun', '134_lie', '135_day', '136_sky', '137_hear',
       '138_head', '139_hair', '140_earth', '141_spit', '142_push',
       '143_leg', '144_dig', '145_play', '146_night', '147_tail',
       '148_smell', '149_i', '150_we', '151_five', '152_fog', '153_knee',
       '154_wash', '155_t

In [35]:
len(words)

203

In [10]:
data_df[data_df['Parameter_ID']=='91_vomit']

,ID,Local_ID,Language_ID,Parameter_ID,Value,Form,Segments,Comment,Source,Cognacy,Loan,Graphemes,Profile,Prosody,Morpheme_Glosses,Partial_Cognacy,Chinese_Characters
0,Beijing-91_vomit-1,NaN,Beijing,91_vomit,tʰu⁵¹,tʰu⁵¹,tʰ u ⁵¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
1,Haerbin-91_vomit-1,NaN,Haerbin,91_vomit,tʰu⁵³,tʰu⁵³,tʰ u ⁵³,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
2,Jinan-91_vomit-1,NaN,Jinan,91_vomit,tʰu³¹,tʰu³¹,tʰ u ³¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐
3,Rongcheng-91_vomit-1,NaN,Rongcheng,91_vomit,ou²¹³⁻³⁵ tʰu²¹⁴,ou²¹³⁻³⁵ tʰu²¹⁴,ou ²¹³ + tʰ u ²¹⁴,copulative synonyme,Liu2007,NaN,NaN,NaN,NaN,n t + i n t,nausea/嘔 spit/吐,2 1,嘔 吐
4,Taiyuan-91_vomit-1,NaN,Taiyuan,91_vomit,tʰu⁵³ lə⁰,tʰu⁵³ lə⁰,tʰ u ⁵³ + l ə ⁰,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t,nausea/嘔 _:PERFECTIVE/了,2 5,嘔 嘞
5,XiAn-91_vomit-1,NaN,XiAn,91_vomit,ŋou²¹ tʰu⁵³,ŋou²¹ tʰu⁵³,ŋ ou ²¹ + tʰ u ⁵³,copulative synonyme,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t,nausea/嘔 spit/吐,2 1,嘔 吐
6,Chengdu-91_vomit-1,NaN,Chengdu,91_vomit,fa²¹⁻⁵⁵ tʰu⁵³ lo²¹,fa²¹⁻⁵⁵ tʰu⁵³ lo²¹,f a ²¹ + tʰ u ⁵³ + l o ²¹,!i,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t + i n t,_send.out/發 spit/吐 _:PERFECTIVE/了,4 1 5,發 吐 了
7,Chengdu-91_vomit-2,NaN,Chengdu,91_vomit,tʰu⁵³⁻⁵⁵ lo²¹,tʰu⁵³⁻⁵⁵ lo²¹,tʰ u ⁵³ + l o ²¹,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t,spit/吐 _:PERFECTIVE/了,1 5,吐 了
8,Chengdu-91_vomit-3,NaN,Chengdu,91_vomit,ŋəu⁵³⁻⁴⁵ lo²¹,ŋəu⁵³⁻⁴⁵ lo²¹,ŋ əu ⁵³ + l o ²¹,!i,Liu2007,NaN,NaN,NaN,NaN,i n t + i n t,nausea/嘔 _:PERFECTIVE/了,2 5,嘔 了
9,Nanjing-91_vomit-1,NaN,Nanjing,91_vomit,tʰu⁴⁴,tʰu⁴⁴,tʰ u ⁴⁴,NaN,Liu2007,NaN,NaN,NaN,NaN,i n t,spit/吐,1,吐


Q1. Do cities that are geographically close have similar characteristics in the same Parameter_ID?

Use word2vec embeddings to test the similarities

Q2 under the same word, test the value/form/segements difference among different areas

But with what method to calculate the phonetics difference? Use value /Form or Segments

Q3 does the similarity of embedding has the positive coefficient with values simialrity

Q4 use the network to show the tree 

Q5 Model to show the dialect distribution 

Q6 N-Gram Analysis 

Q7 the same city such as Chengdu ,Nnajing and so on has varieties , what is the reason for this variety 

step3 Raise some questions related with the dialetcs' geography and explore it 